In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Merchandise_Popularity_Prediction/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Merchandise_Popularity_Prediction/Test.csv')

In [ ]:
df = train.append(test)

In [ ]:
# df.columns

In [ ]:
x = pd.get_dummies(df.Category_1)
x.columns = ['Cat_1_'+str(i) for i in x.columns]
x.head()
df = pd.concat([df,x],axis=1)

In [ ]:
x = pd.get_dummies(df.Category_2)
x.columns = ['Cat_2_'+str(i) for i in x.columns]
x.head()
df = pd.concat([df,x],axis=1)

In [ ]:
df.drop(columns=['Category_1','Category_2'],axis=1,inplace=True)

In [ ]:
df.shape,train.shape,test.shape

((30348, 24), (18208, 12), (12140, 11))

In [ ]:
train = df.head(len(train))
test = df.tail(len(test))

In [ ]:
test.shape,train.shape

((12140, 24), (18208, 24))

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2)

In [ ]:
!pip install catboost

     |████████████████████████████████| 65.8MB 56kB/s 


In [ ]:
train.popularity.unique()

array([4., 5., 3., 1., 0.])

In [ ]:
np.unique(y_train)

array([0., 1., 3., 4., 5.])

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         depth=6,
                         random_seed=121, 
                         iterations=3500, 
                         learning_rate=0.1,
                         leaf_estimation_iterations=1,
                         l2_leaf_reg=1,
                         bootstrap_type='Bayesian', 
                         bagging_temperature=1, 
                         random_strength=1,
                         od_type='Iter', 
                         border_count=100,
                         od_wait=500)
cat.fit(X_train, y_train, verbose=100,
        use_best_model=True,
        eval_set=[(X_train, y_train),(X_valid, y_valid)],
        plot=False)
predictions = cat.predict(X_valid)
print('accuracy:', f1_score(y_valid, predictions, average='weighted'))

0:	learn: 0.7647767	test: 0.7647767	test1: 0.7558963	best: 0.7558963 (0)	total: 69.6ms	remaining: 4m 3s
100:	learn: 0.8072864	test: 0.8072864	test1: 0.7734167	best: 0.7734167 (100)	total: 1.99s	remaining: 1m 7s
200:	learn: 0.8495233	test: 0.8495233	test1: 0.7950686	best: 0.7952494 (194)	total: 3.85s	remaining: 1m 3s
300:	learn: 0.8772146	test: 0.8772146	test1: 0.8080398	best: 0.8080398 (300)	total: 5.72s	remaining: 1m
400:	learn: 0.9001486	test: 0.9001486	test1: 0.8195328	best: 0.8201019 (388)	total: 7.64s	remaining: 59s
500:	learn: 0.9137617	test: 0.9137617	test1: 0.8267607	best: 0.8275127 (491)	total: 9.54s	remaining: 57.1s
600:	learn: 0.9256613	test: 0.9256613	test1: 0.8332441	best: 0.8338890 (574)	total: 11.5s	remaining: 55.2s
700:	learn: 0.9352595	test: 0.9352595	test1: 0.8368057	best: 0.8369975 (696)	total: 13.4s	remaining: 53.4s
800:	learn: 0.9446565	test: 0.9446565	test1: 0.8410802	best: 0.8413392 (794)	total: 15.3s	remaining: 51.5s
900:	learn: 0.9522827	test: 0.9522827	test1: 

In [ ]:
print('accuracy:', f1_score(y_valid, predictions, average='weighted'))

accuracy: 0.857935126308043


In [ ]:
Xtest = test

In [ ]:
from sklearn.model_selection import KFold

errcat = []
y_pred_totcat = []

fold = KFold(n_splits=10, shuffle=True, random_state=101)

for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    cat = CatBoostClassifier(loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         depth=6,
                         random_seed=121, 
                         iterations=3500, 
                         learning_rate=0.1,
                         leaf_estimation_iterations=1,
                         l2_leaf_reg=1,
                         bootstrap_type='Bayesian', 
                         bagging_temperature=0.8, 
                         random_strength=1,
                         od_type='Iter', 
                         border_count=100,
                         od_wait=500)
    cat.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=200)

    y_pred_cat = cat.predict(X_test)
    print("Accuracy: ", f1_score(y_test,y_pred_cat, average='weighted'))

    errcat.append(f1_score(y_test,y_pred_cat, average='weighted'))
    p = cat.predict_proba(Xtest)
    y_pred_totcat.append(p)

Accuracy:  0.8689226212502722
Accuracy:  0.863282058606027
Accuracy:  0.8491480139975321
Accuracy:  0.8503272635354306
Accuracy:  0.8645094428274402
Accuracy:  0.866349389624819
Accuracy:  0.8586177976977565
Accuracy:  0.8523310058370366
Accuracy:  0.8661493507692198
Accuracy:  0.8589267017535376


In [ ]:
np.shape(y_pred_totcat)

(10, 12140, 5)

In [ ]:
preds1 =[]
for j in range(len(y_pred_totcat[0])):
  preds1.append(y_pred_totcat[0][j])
np.shape(preds1)

(12140, 5)

In [ ]:
sub = pd.DataFrame(preds1)
sub.head()


# 0.41127

,0,1,2,3,4
0,9.980620e-06,0.019442,0.169421,0.784624,0.026503
1,2.184602e-07,0.001261,0.020919,0.977366,0.000454
2,2.696347e-07,0.000111,0.020996,0.966037,0.012856
3,3.158135e-07,0.000721,0.003692,0.984762,0.010825
4,2.987975e-07,0.000059,0.000854,0.998731,0.000356


In [ ]:
preds2 =[]
for j in range(len(y_pred_totcat[1])):
  preds2.append(y_pred_totcat[1][j])
np.shape(preds2)


(12140, 5)

In [ ]:
sub = pd.DataFrame(preds2)
sub.head()
sub.to_csv('preds2.csv',index=False)
# 0.41833

In [ ]:
preds3 =[]
for j in range(len(y_pred_totcat[2])):
  preds3.append(y_pred_totcat[2][j])
np.shape(preds3)
sub = pd.DataFrame(preds3)
sub.head()
sub.to_csv('preds3.csv',index=False)
# 0.40874

In [ ]:
preds4 =[]
for j in range(len(y_pred_totcat[3])):
  preds4.append(y_pred_totcat[3][j])
np.shape(preds4)
sub = pd.DataFrame(preds4)
sub.head()
sub.to_csv('preds4.csv',index=False)

# 0.41837

In [ ]:
preds5 =[]
for j in range(len(y_pred_totcat[4])):
  preds5.append(y_pred_totcat[4][j])
np.shape(preds5)
sub = pd.DataFrame(preds5)
sub.head()
sub.to_csv('preds5.csv',index=False)

preds6 =[]
for j in range(len(y_pred_totcat[5])):
  preds6.append(y_pred_totcat[5][j])
np.shape(preds6)
sub = pd.DataFrame(preds6)
sub.head()
sub.to_csv('preds6.csv',index=False)

# 0.40638

preds7 =[]
for j in range(len(y_pred_totcat[6])):
  preds7.append(y_pred_totcat[6][j])
np.shape(preds7)
sub = pd.DataFrame(preds7)
sub.head()
sub.to_csv('preds7.csv',index=False)

preds8 =[]
for j in range(len(y_pred_totcat[7])):
  preds8.append(y_pred_totcat[7][j])
np.shape(preds8)
sub = pd.DataFrame(preds8)
sub.head()
sub.to_csv('preds8.csv',index=False)



In [ ]:
preds9 =[]
for j in range(len(y_pred_totcat[8])):
  preds9.append(y_pred_totcat[8][j])
np.shape(preds9)
sub = pd.DataFrame(preds9)
sub.head()
sub.to_csv('preds9.csv',index=False)

# 0.41219



preds10 =[]
for j in range(len(y_pred_totcat[9])):
  preds10.append(y_pred_totcat[9][j])
np.shape(preds10)
sub = pd.DataFrame(preds10)
sub.head()
sub.to_csv('preds10.csv',index=False)
